<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/Explode%2CSplit%2Carray_contains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=21d22a5da9649fc23a7ed967c55c1b1abe7532b4638ee87dd9c1fe275c04e7dd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.sql.types import StringType,StructField,StructType,IntegerType,ArrayType
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Explode_split_array_contains').getOrCreate()

## Agenda

- Explode()
- Split()
- Array()
- Array_contains()

## - Explode()

Use explode() function to create a new now for each element in the given array column.

In [20]:
from pyspark.sql.functions import col,explode
from pyspark.sql.functions import col,lit,when,expr

In [6]:
data = [
    ('apple',[1,2],{'shop_A' : 'Mumbai','shop_B':'kolatha'}),\
    ('orange',[2,3],{'shop_A' : 'Kerala','shop_B':'banglroe'}),\
    ('grapes',[4,5],{'shop_A' : 'Chennai','shop_B':'Pune'})
]

In [7]:
schema = StructType([\
                     StructField('Fruits',StringType()),\
                     StructField('No_of_kgs',ArrayType(IntegerType())),\
                     StructField('Head_quaters',StringType())
                     ])

df = spark.createDataFrame(data,['Fruits','No_of_kgs','Head_quaters'])

In [8]:
df.printSchema()

root
 |-- Fruits: string (nullable = true)
 |-- No_of_kgs: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Head_quaters: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [9]:
df.show(truncate=False)

+------+---------+--------------------------------------+
|Fruits|No_of_kgs|Head_quaters                          |
+------+---------+--------------------------------------+
|apple |[1, 2]   |{shop_B -> kolatha, shop_A -> Mumbai} |
|orange|[2, 3]   |{shop_B -> banglroe, shop_A -> Kerala}|
|grapes|[4, 5]   |{shop_B -> Pune, shop_A -> Chennai}   |
+------+---------+--------------------------------------+



In [27]:
data = [('apple',[1,2],['Banglore','Austin']),\
        ('orange',[4,5],['Chennai','New_york']),\
        ('grapes',[7,8],['Kochi','DC'])]

schema = StructType([\
                     StructField('fruits',StringType()),\
                     StructField('No_of_kgs',ArrayType(IntegerType())),\
                     StructField('Metro_city',ArrayType(StringType()))
                     ]) #Interger present inside the array type

df = spark.createDataFrame(data,schema)
df1 = df.withColumn('explode_col',explode(col('No_of_kgs'))).select('fruits','explode_col')
print('Before explode')
df.show()
print('after explode')
df1.show()


Before explode
+------+---------+-------------------+
|fruits|No_of_kgs|         Metro_city|
+------+---------+-------------------+
| apple|   [1, 2]| [Banglore, Austin]|
|orange|   [4, 5]|[Chennai, New_york]|
|grapes|   [7, 8]|        [Kochi, DC]|
+------+---------+-------------------+

after explode
+------+-----------+
|fruits|explode_col|
+------+-----------+
| apple|          1|
| apple|          2|
|orange|          4|
|orange|          5|
|grapes|          7|
|grapes|          8|
+------+-----------+



In [29]:
## Multiple column explode()


df.withColumn('total_kgs',explode('No_of_kgs'))\
  .withColumn('Metro_',explode('Metro_city'))\
  .withColumn('Metro_',explode('Metro_city'))\
  .select('fruits','total_kgs','Metro_').show()

+------+---------+--------+
|fruits|total_kgs|  Metro_|
+------+---------+--------+
| apple|        1|Banglore|
| apple|        1|  Austin|
| apple|        1|Banglore|
| apple|        1|  Austin|
| apple|        2|Banglore|
| apple|        2|  Austin|
| apple|        2|Banglore|
| apple|        2|  Austin|
|orange|        4| Chennai|
|orange|        4|New_york|
|orange|        4| Chennai|
|orange|        4|New_york|
|orange|        5| Chennai|
|orange|        5|New_york|
|orange|        5| Chennai|
|orange|        5|New_york|
|grapes|        7|   Kochi|
|grapes|        7|      DC|
|grapes|        7|   Kochi|
|grapes|        7|      DC|
+------+---------+--------+
only showing top 20 rows



In [12]:
df1 = df.withColumn('shop_1',col('No_of_kgs').getItem(0))\
        .withColumn('shop_2',col('No_of_kgs').getItem(1))
df1.show()

+------+---------+------+------+
|fruits|No_of_kgs|shop_1|shop_2|
+------+---------+------+------+
| apple|   [1, 2]|     1|     2|
|orange|   [4, 5]|     4|     5|
|grapes|   [7, 8]|     7|     8|
+------+---------+------+------+



In [50]:
data = [(1,'maheer',['dotnet','azure']),
        (2,'wafa',['java','aws']),
        (3,'Micheal',['aws','pyhton'])]

schema = StructType([\
                     StructField('Emp_id',IntegerType()),\
                     StructField('Emp_name',StringType()),\
                     StructField('skills',ArrayType(StringType()))
                     ])

df = spark.createDataFrame(data,schema)
df.show()
spark.createDataFrame(data,schema).printSchema()

+------+--------+---------------+
|Emp_id|Emp_name|         skills|
+------+--------+---------------+
|     1|  maheer|[dotnet, azure]|
|     2|    wafa|    [java, aws]|
|     3| Micheal|  [aws, pyhton]|
+------+--------+---------------+

root
 |-- Emp_id: integer (nullable = true)
 |-- Emp_name: string (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Split()

Split() function returns an array type after splitting the string column by delimiter.

In [42]:
from pyspark.sql.functions import split

In [32]:
data = [\
        (1,'Micheal','.net,azure,sql'),
        (2,'Daniel','java,aws,sql')]
schema = ['Emp_id','Emp_name','skills']
df = spark.createDataFrame(data,schema)
df.show()


+------+--------+--------------+
|Emp_id|Emp_name|        skills|
+------+--------+--------------+
|     1| Micheal|.net,azure,sql|
|     2|  Daniel|  java,aws,sql|
+------+--------+--------------+



In [40]:
df.withColumn('skill_set',split(col('skills'),',')).select('Emp_id','Emp_name','skill_set').show()

+------+--------+------------------+
|Emp_id|Emp_name|         skill_set|
+------+--------+------------------+
|     1| Micheal|[.net, azure, sql]|
|     2|  Daniel|  [java, aws, sql]|
+------+--------+------------------+



## Array()


Use array() function to create a new array column by merging the data from multiple columns

In [41]:
from pyspark.sql.functions import array

In [44]:
data = [\
        (1,'Micheal','azure','sql'),
        (2,'Daniel','aws','python')]
schema = ['Emp_id','Emp_name','primary_skills','secondary_skills']
df = spark.createDataFrame(data,schema)
df.show()


+------+--------+--------------+----------------+
|Emp_id|Emp_name|primary_skills|secondary_skills|
+------+--------+--------------+----------------+
|     1| Micheal|         azure|             sql|
|     2|  Daniel|           aws|          python|
+------+--------+--------------+----------------+



In [45]:
df.withColumn('skills',array(col('primary_skills'),col('secondary_skills'))).select('Emp_id','Emp_name','skills').show()

+------+--------+-------------+
|Emp_id|Emp_name|       skills|
+------+--------+-------------+
|     1| Micheal| [azure, sql]|
|     2|  Daniel|[aws, python]|
+------+--------+-------------+



## Array_contain()

This function used to check if array column contain a value. Return Null if the array is null , True if the array contain the value and False otherwise

In [48]:
from pyspark.sql.functions import array_contains

In [51]:
df.show()

+------+--------+---------------+
|Emp_id|Emp_name|         skills|
+------+--------+---------------+
|     1|  maheer|[dotnet, azure]|
|     2|    wafa|    [java, aws]|
|     3| Micheal|  [aws, pyhton]|
+------+--------+---------------+



In [52]:
df.withColumn('has_AWS_skills',array_contains(col('skills'),'aws')).show()

+------+--------+---------------+--------------+
|Emp_id|Emp_name|         skills|has_AWS_skills|
+------+--------+---------------+--------------+
|     1|  maheer|[dotnet, azure]|         false|
|     2|    wafa|    [java, aws]|          true|
|     3| Micheal|  [aws, pyhton]|          true|
+------+--------+---------------+--------------+

